In [1]:
import os
import sys

import numpy as np
import pandas as pd
import openslide
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('..'))
from utils.utils_data import get_veesel_sheets, get_segmentations
from utils.utils_geometry import get_contours

from utils.utils_vis import plot_artery_ann, save_image
from utils.utils_constants import (CLASSIFICATION_SEVERITY_MAPPING, 
                                   VESSEL_NEPTUNE_PAT_INFO_PATH as VESSEL_PAT_INFO_PATH, 
                                   CLASSIFICATION_PATH, SEGMENTATION_DIR, 
                                   TRI_CASE_DIR, CROPPED_VESSELS_DIR)

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
pat_df = pd.read_csv(VESSEL_PAT_INFO_PATH)

available_sheetnames = pd.ExcelFile(CLASSIFICATION_PATH).sheet_names

for i, (_, row_slide) in enumerate(pat_df.iterrows()):
    slide_filename = row_slide["WSI_Selected"]
    slide_basename = os.path.splitext(slide_filename)[0]

    logging.info(f"Processing: {i+1}/{len(pat_df)} {slide_filename}")
    slide_path = os.path.join(TRI_CASE_DIR, slide_filename)
    slide = openslide.OpenSlide(slide_path)

    classifications = get_veesel_sheets(CLASSIFICATION_PATH, slide_basename, available_sheetnames, remove_others=False)
    if classifications.empty:
        continue  # Skip to if no relevant data
    classifications['Arteriosclerosis Severity'] = classifications['Arteriosclerosis Severity'].map(CLASSIFICATION_SEVERITY_MAPPING)

    segmentations_path = os.path.join(SEGMENTATION_DIR, f"{slide_basename}.geojson")
    segmentations = get_segmentations(segmentations_path, clean=True)

    for i, (_, row_vessel)in enumerate(classifications.iterrows()):
        if row_vessel["Image Name"] != "Biopsy_006_WSI_001_A04_96296_29979_400_868.png": continue
        bbox_x, bbox_y, bbox_width, bbox_height = map(int, row_vessel["Bounding Box"].split(","))  
        try:
            cnt_outer, cnts_middle, cnts_inner, cnts_hys = get_contours(segmentations, slide_basename,
                                                                        (bbox_x, bbox_y, bbox_width, bbox_height))
        except Exception as e:
            print(e)
            img = np.array(slide.read_region((bbox_x, bbox_y), 0, (bbox_width, bbox_height)).convert("RGB"))
            # img_w_ann = plot_sartery_ann(img, cnt_outer, cnts_middle, cnts_inner, cnts_hys)
            plt.imshow(img)
            plt.show()

        # path_img_to_save = os.path.join(CROPPED_VESSELS_DIR, row["Artery Type"], 
        #                                 row["Image Name"].replace(".png", "_ori.png"))
        # # save_image(img, path_img_to_save)
        # img_w_ann = plot_artery_ann(img, cnt_outer, cnts_middle, cnts_inner, cnts_hys)
        # # save_image(img_w_ann, path_img_to_save.replace("_ori.png", "_w_ann.png"))
        # # mask_w_hyalinosis = plot_artery_ann(np.zeros(img.shape, dtype=np.uint8), cnt_outer, cnts_middle, cnts_inner, cnts_hys, -1)
        # # save_image(mask_w_hyalinosis, path_img_to_save.replace("_ori.png", "_mask.png"))
        # mask_wo_hyalinosis = plot_artery_ann(np.zeros(img.shape, dtype=np.uint8), cnt_outer, cnts_middle, cnts_inner, [], -1)
        # save_image(mask_wo_hyalinosis, path_img_to_save.replace("_ori.png", "_mask_wo_hya.png"))

2024-08-10 18:38:34,213 - INFO - Processing: 1/247 10_26609_000_002_L2_TRI.svs
2024-08-10 18:38:36,040 - INFO - Processing: 2/247 11_26609_000_009_L2_TRI.svs
2024-08-10 18:38:41,657 - INFO - Processing: 3/247 11_26609_000_012_L02_TRI.svs
2024-08-10 18:38:43,952 - INFO - Processing: 4/247 11_26609_000_013_L02 TRI.svs
2024-08-10 18:38:45,241 - INFO - Processing: 5/247 0_784_A_0047801.svs
2024-08-10 18:38:48,460 - INFO - Processing: 6/247 0_796_A_0047806.svs
2024-08-10 18:38:55,234 - INFO - Processing: 7/247 0_800_A_0047811.svs
2024-08-10 18:38:57,258 - INFO - Processing: 8/247 11_26609_000_503_L2_TRI.svs
2024-08-10 18:38:59,926 - INFO - Processing: 9/247 11_26609_001_004 L7 TRI.ndpi
2024-08-10 18:39:00,636 - INFO - Processing: 10/247 12_26609_001_011 LUNK5 TRI.ndpi
2024-08-10 18:39:00,946 - INFO - Sheet 12_26609_001_011 LUNK5 TRI not found in the classifications file.
2024-08-10 18:39:00,949 - INFO - Processing: 11/247 13_26609_001_013 LUNK1 TRI.ndpi
2024-08-10 18:39:02,241 - INFO - Proc